# Table 8: Matched-$R_0$ Comparison execution

This notebook runs the **matched-$R_0$ comparison** between $L_2$ regularization and VIB to populate Table 8 in the paper.

**Estimated runtime**: 4-5 hours on CPU, ~1 hour on GPU

---

## 0. Colab Upload (Optional)
If you are running on Google Colab, use the cell below to upload the `occam-hedge-table8-lean.zip` file.

In [ ]:
try:
    from google.colab import files
    import os
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
        !unzip -o {fn}
except ImportError:
    print("Not running in Google Colab. Skipping upload step.")

## 1. Environment Setup

In [ ]:
import os
import sys
import json
import torch
import pandas as pd
import numpy as np
from pathlib import Path

# Add src to path
ROOT = Path(os.getcwd()).resolve()
if (ROOT / "src").exists():
    sys.path.append(str(ROOT / "src"))
    print("✅ Added src/ to sys.path")
else:
    print("❌ src/ directory not found. Please run from project root or ensure files unzipped correctly.")

print(f"PyTorch Version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 2. Config Verification

In [ ]:
CONFIG_PATH = ROOT / "configs" / "paper_run.json"
with open(CONFIG_PATH, 'r') as f:
    paper_cfg = json.load(f)

print("Canonical Config (paper_run.json):")
print(f"  n_steps: {paper_cfg['n_steps']}")
print(f"  T: {paper_cfg['T']}")
print(f"  gamma: {paper_cfg['gamma']}")
print(f"  train_eta: {paper_cfg['train_eta']}")

## 3. Run Experiment

This will execute the full 3-seed sweep. If you want a quick test, edit `scripts/run_regularization_control.py` to set `"n_seeds": 1`.

In [ ]:
!PYTHONPATH=./src python scripts/run_regularization_control.py | tee results/table8_execution.log

## 4. Extract Results for Table 8

In [ ]:
summary_path = ROOT / "results" / "regularization_control_summary.csv"
df_summary = pd.read_csv(summary_path)

# 1. Get VIB baseline
vib = df_summary[df_summary['model_type'] == 'VIB'].iloc[0]
vib_r0 = vib['R0_mean']

print(f"VIB Baseline R0: {vib_r0:.4f}")

# 2. Find best L2 match
l2_results = df_summary[df_summary['model_type'] == 'L2'].copy()
l2_results['r0_diff'] = (l2_results['R0_mean'] - vib_r0).abs()
best_l2 = l2_results.loc[l2_results['r0_diff'].idxmin()]

print(f"\nBest L2 Match:")
print(f"  l2_lambda: {best_l2['l2_lambda']}")
print(f"  Matched R0: {best_l2['R0_mean']:.4f}")
print(f"  R1: {best_l2['R1_matched_mean']:.4f}")
print(f"  Deg: {best_l2['R1_matched_mean'] / best_l2['R0_mean']:.2f}")
print(f"  ProbeAUC: {best_l2['probe_auc_mean']:.3f}")

## 5. Generate LaTeX Snippet

In [ ]:
erm = l2_results[l2_results['l2_lambda'] == 0.0].iloc[0]

print("LaTeX Copy-Paste Snippet:")
print("-" * 40)
print(f"ERM & -- & {erm['R0_mean']:.3f} & {erm['R1_matched_mean']:.3f} & {erm['R1_matched_mean']/erm['R0_mean']:.2f} & {erm['probe_auc_mean']:.2f} \\\\")
print(f"$L_2$ & $\\lambda={best_l2['l2_lambda']}$ & {best_l2['R0_mean']:.3f} & {best_l2['R1_matched_mean']:.3f} & {best_l2['R1_matched_mean']/best_l2['R0_mean']:.2f} & {best_l2['probe_auc_mean']:.2f} \\\\")
print(f"VIB (Occam's Hedge) & $\\beta=0.5$ & {vib['R0_mean']:.3f} & {vib['R1_matched_mean']:.3f} & {vib['R1_matched_mean']/vib['R0_mean']:.2f} & {vib['probe_auc_mean']:.2f} \\\\")
print("-" * 40)